In [1]:
using Symbolics
using RuntimeGeneratedFunctions
RuntimeGeneratedFunctions.init(@__MODULE__)

In [ ]:
_prod(a, b, c, arrs...) = a .* _prod(b, c, arrs...)
_prod(a, b) = a .* b
_prod(a) = a

" makes polynomial combinations of basis "
function hamiltonian_poly(z, order, inds...)
    ham = []

    if order == 0
        Num(1)
    elseif order == length(inds)
        ham = vcat(ham, _prod([z[i] for i in inds]...))
    else
        start_ind = length(inds) == 0 ? 1 : inds[end]
        for j in start_ind:length(z)
            ham = vcat(ham, hamiltonian_poly(z, order, inds..., j))
        end
    end

    return ham
end

In [ ]:
" collects and sums only polynomial combinations of basis "
function hamiltonian(z, a, order)
    ham = []

    for i in 1:order
        ham = vcat(ham, hamiltonian_poly(z, i))
    end

    sum(collect(a .* ham))
end

In [ ]:
" collects and sums polynomial and trignometric combinations of basis "
function hamil_trig(z, a, order, trig_wave_num)
    ham = []

    # Polynomial basis
    for i in 1:order
        ham = vcat(ham, hamiltonian_poly(z, i))
    end

    # Trignometric basis
    for k = 1:trig_wave_num
        ham = vcat(ham, vcat(sin.(k*z)), vcat(cos.(k*z)))
    end

    ham = sum(collect(a .* ham))

    return ham

end

In [ ]:
" collects and sums polynomial, trignometric, and states differences combinations of basis "
function hamiltonian_two(z, a, order, trig_wave_num, diffs_power)
    ham = []

    # Polynomial basis
    for i in 1:order
        ham = vcat(ham, hamiltonian_poly(z, i))
    end

    # Trignometric basis
    for k = 1:trig_wave_num
        ham = vcat(ham, vcat(sin.(k*z)), vcat(cos.(k*z)))
    end

    if diffs_power > 0
        # States difference basis
        diffs = Vector{Num}()
        idx = 1
        for i in 1:length(z)
            for j in 1:length(z)
                if i == j
                    continue  # skip index where difference is between same state
                end
                push!(diffs, (z[i] - z[j]))
                idx += 1
            end
        end

        for k = 1:diffs_power
            ham = vcat(ham, vcat(diffs .^ k))
        end
    end

    ham = sum(collect(a .* ham))
    return ham

end

In [ ]:
"""
returns the number of required parameters
depending on whether there are trig basis or not
"""
function calculate_nparams(nd, polyorder, trig_wave_num, diffs_power)
    # binomial used to get the combination of polynomials till the highest order without repeat, e.g nparam = 34 for 3rd order, with z = q,p each of 2 dims
    # nd: total number of dims of all variable states
    nparam = binomial(nd + polyorder, polyorder) - 1

    if trig_wave_num > 0
        # first 2 in the product formula b/c the trig basis are sin and cos i.e. two basis functions
        nparam += 2 * trig_wave_num * nd
    end

    if diffs_power > 0
        # diffs power is the 
        nparam += diffs_power * nd * (nd-1)
    end

    return nparam
end

In [ ]:

# # symbolic variables
# @variables a[1:nparam]
# @variables q[1:d]
# @variables p[1:d]
# z = vcat(q,p)

# diffs = Vector{Num}()
# idx = 1
# for i in 1:length(z)
#     for j in 1:length(z)
#         if i == j
#             continue  # skip index where difference is between same state
#         end
#         push!(diffs, (z[i] - z[j]))
#         idx += 1
#     end
# end
# println(diffs)

In [ ]:
" returns a function that can build the gradient of the hamiltonian "
function ΔH_func_builder_two(d, polyorder, trig_wave_num, diffs_power)
    # nd is the total number of dimensions of all the states, e.g. if q,p each of 3 dims, that is 6 dims in total
    nd = 2d
    
    # binomial used to get the combination of variables till the highest order without repeat, nparam = 34 for 3rd order, with z = q,p each of 2 dims
    nparam = calculate_nparams(nd, polyorder, trig_wave_num, diffs_power)

    # symbolic variables
    @variables a[1:nparam]
    @variables q[1:d]
    @variables p[1:d]
    z = vcat(q,p)
    Dz = Differential.(z)
    
    # make a basis library
    ham = hamiltonian_two(z, a, polyorder, trig_wave_num, diffs_power)
    
    # gives derivative of the hamiltonian, but not the skew-symmetric true one
    f = [expand_derivatives(dz(ham)) for dz in Dz]

    # line below makes the vector into a hamiltonian vector field by multiplying with the skew-symmetric matrix
    ∇H = vcat(f[d+1:2d], -f[1:d])

    # builds a function that calculates Hamiltonian gradient and converts the function to a native Julia function
    ∇H_eval = @RuntimeGeneratedFunction(Symbolics.inject_registered_module_functions(build_function(∇H, z, a)[2]))
    
    return ∇H_eval

end


In [ ]:

" returns a function that can build the gradient of the hamiltonian "
function hamilGrad_func_builder(d, polyorder, trig_wave_num)
    # nd is the total number of dimensions of all the states, e.g. if q,p each of 3 dims, that is 6 dims in total
    nd = 2d
    # binomial used to get the combination of variables till the highest order without repeat, nparam = 34 for 3rd order, with z = q,p each of 2 dims
    nparam = calculate_nparams(d, polyorder, trig_wave_num)

    # symbolic variables
    @variables a[1:nparam]
    @variables q[1:d]
    @variables p[1:d]
    z = vcat(q,p)

    # usesine: whether to add trig basis or not
    if trig_wave_num > 0

        # gives derivative of the hamiltonian, but not the skew-symmetric true one
        Dz = Differential.(z)
        ∇H_add_trig = [expand_derivatives(dz(hamil_trig(z, a, polyorder, trig_wave_num))) for dz in Dz]

        # line below makes the vector into a hamiltonian vector field by multiplying with the skew-symmetric matrix
        ∇H_trig = vcat(∇H_add_trig[d+1:2d], -∇H_add_trig[1:d])

        # builds a function that calculates Hamiltonian gradient and converts the function to a native Julia function
        ∇H_eval = @RuntimeGeneratedFunction(Symbolics.inject_registered_module_functions(build_function(∇H_trig, z, a)[2]))

        return ∇H_eval

    else

        # gives derivative of the hamiltonian, but not the skew-symmetric true one
        Dz = Differential.(z)
        f = [expand_derivatives(dz(hamiltonian(z, a, polyorder))) for dz in Dz]

        # line below makes the vector into a hamiltonian by multiplying with the skew-symmetric matrix
        ∇H = vcat(f[d+1:2d], -f[1:d])

        # builds a function that calculates Hamiltonian gradient and converts the function to a native Julia function
        ∇H_eval = @RuntimeGeneratedFunction(Symbolics.inject_registered_module_functions(build_function(∇H, z, a)[2]))
        
        return ∇H_eval

    end

end

#### TESTING TIME ####

In [ ]:
# nd: total dims of all variables i.e p,q
nd = 4
# since we always have q and p, i.e 2 variables only, d will always be nd/2
d = div(nd, 2)

" trig_wave_num can be adjusted if higher frequency arguments expected "
trig_wave_num = 1

# highest order of polynomial library function
polyorder = 3

diffs_power = 1
"binomial used to get the combination of variables till the highest 
order without repeat, e.g with usesine= false, nparam = 34 for 3rd 
order, with z = q,p each of 2 dims"

nparam = calculate_nparams(nd, polyorder, trig_wave_num, diffs_power)
println(nparam)

In [ ]:
# builds a function that calculates Hamiltonian gradient and converts the function to a native Julia function
∇H_eval = ΔH_func_builder_two(nd, polyorder, trig_wave_num, diffs_power)

In [ ]:
# wrapper function for generalized SINDY hamiltonian gradient
function hamilGradient_general!(out, z, a::AbstractVector{T}, t) where T
    ∇H_eval(out, z, a)
    return out
end

In [ ]:
# 2D system with 4 variables [q₁, q₂, p₁, p₂] where q₂ = 0 and p₂ = 0
nd = 4

# 2 dims each of p and q gives 2*d = 4 variables
out = zeros(nd)

# let (a) be a vector of zeros initially of length 34 (b/c 34 is number of poly combinations for 2 variables, with 2 dims of highest order 3)

##################### NOTE: IN ACTUAL SCRIPT WE INITIALIZE THIS TO ZERO TO ALLOW BETTER OPTIMIZATION
a = ones(nparam)

x₀ = [2, 0, 0, 0]

t = 0
    

In [ ]:
hamilGradient_general!(out, x₀, a, 0)

#### VISUALIZE (a) coefficients ####

In [ ]:
polyorder = 3
trig_wave_num = 1
diffs_power = 1
nd = 4

# since we always have q and p, i.e 2 variables only, d will always be nd/2
d = div(nd, 2)
nparam = calculate_nparams(nd, polyorder, trig_wave_num, diffs_power)
# symbolic variables
@variables a[1:nparam]
@variables q[1:d]
@variables p[1:d]
z = vcat(q,p)

ham = []

# Polynomial basis
for i in 1:polyorder
    ham = vcat(ham, hamiltonian_poly(z, i))
end

# Trignometric basis
for k = 1:trig_wave_num
    ham = vcat(ham, vcat(sin.(k*z)), vcat(cos.(k*z)))
end

if diffs_power > 0
    # States difference basis
    diffs = Vector{Num}()
    idx = 1
    for i in 1:length(z)
        for j in 1:length(z)
            if i == j
                continue  # skip index where difference is between same state
            end
            push!(diffs, (z[i] - z[j]))
            idx += 1
        end
    end

    for k = 1:diffs_power
        ham = vcat(ham, vcat(diffs .^ k))
    end
end

ham = (collect(a .* ham))